In [1]:
# Import packages
import pandas as pd
import re

# Set  number format
# np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = "{:,.2f}".format


#  ixi MRIO

## 2019 Data

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
# ---------------------------------------------------------
dir_path = "../data/exiobase/_ixi/IOT_2019_ixi/"


# ---------------------------------------------------------
# IMPORT MATRICES
# ---------------------------------------------------------
# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL
# ---------------------------------------------------------
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()

In [3]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


Index(['Manure treatment (biogas), storage and land application',
       'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Extraction, liquefaction, and regasification of other petroleum and gaseous materials',
       'Production of electricity by gas',
       'Manufacture of gas; distribution of gaseous fuels through mains',
       'Biogasification of food waste, incl. land application',
       'Biogasification of paper, incl. land application',
       'Biogasification of sewage slugde, incl. land application'],
      dtype='object', name='sector')

In [4]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Extraction of natural gas and services related to natural gas extraction, excluding surveying",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Extraction, liquefaction, and regasification of other petroleum and gaseous materials",
    "Transport via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


### Inter industry

In [8]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)

total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Z)

res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   10,227.76
        Manufacture of gas; distribution of gaseous fuels through mains                                      7.53
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials                0.00
        Transport via pipelines                                                                            631.60
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials          

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
IT                                                  52.02                                             \
CZ                                                  14.39                                              
HU                                                  13.14                                              
LT                                                   6.48                                              
BG                                                   6.18                                              
LV                                                   1.93                                              
FI                                                   1.80                                              
EE                                                   1.20                                              
FR                                                   1.13                                              
GR                                                   0.48                                              
ES                                                   0.25                                              
SE                                                   0.20                                              
PL                                                   0.17                                              
DE                                                   0.14                                              
BE                                                   0.13                                              
NL                                                   0.10                                              
SI                                                   0.09                                              
DK                                                   0.06                                              
RO                                                   0.03                                              
PT                                                   0.03                                              
SK                                                   0.03                                              
MT                                                   0.02                                              
CY                                                   0.01                                              
AT                                                   0.00                                              
HR                                                   0.00                                              
LU                                                   0.00                                              
IE                                                   0.00                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
IT                                                   9.44               \
CZ                                                   5.14                
HU                                                   5.24                
LT                                                   3.26                
BG                                                   1.38                
LV                                                   1.74                
FI                                                  10.49                
EE                                                   3.19                
FR                                                   5.30        

### final demand

In [6]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)

total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Y)

res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   3,175.29
        Manufacture of gas; distribution of gaseous fuels through mains                                     4.25
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials               0.00
        Transport via pipelines                                                                             2.45
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials             0

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
SK                                                  41.73                                             \
GR                                                  22.75                                              
BG                                                  17.70                                              
RO                                                   9.61                                              
SI                                                   2.60                                              
DE                                                   1.16                                              
HU                                                   0.84                                              
FR                                                   0.58                                              
IT                                                   0.54                                              
MT                                                   0.47                                              
LV                                                   0.30                                              
NL                                                   0.30                                              
BE                                                   0.29                                              
PL                                                   0.26                                              
DK                                                   0.22                                              
IE                                                   0.14                                              
EE                                                   0.11                                              
SE                                                   0.09                                              
ES                                                   0.09                                              
FI                                                   0.07                                              
LT                                                   0.05                                              
CZ                                                   0.03                                              
LU                                                   0.02                                              
HR                                                   0.02                                              
AT                                                   0.01                                              
CY                                                   0.01                                              
PT                                                   0.00                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
SK                                                   2.21               \
GR                                                   1.59                
BG                                                   2.32                
RO                                                   4.46                
SI                                                   0.21                
DE                                                  22.75                
HU                                                   2.10                
FR                                                   5.99                
IT                                                   4.35        

### Inter-industry AND final demand

In [7]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY + FINAL DEMAND)
EU_imports_ZY = (
    Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
    + Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
)
totalEU_imports_ZY = EU_imports_ZY.sum(axis=1)

print(totalEU_imports_ZY)

res = EU_imports_ZY.divide(totalEU_imports_ZY, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   13,020.96
        Manufacture of gas; distribution of gaseous fuels through mains                                     11.17
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials                0.00
        Transport via pipelines                                                                            190.86
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials          

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
IT                                                  49.12                                             \
SK                                                  10.21                                              
CZ                                                   9.98                                              
HU                                                   8.47                                              
GR                                                   5.99                                              
BG                                                   4.37                                              
RO                                                   2.44                                              
LT                                                   1.95                                              
LV                                                   1.91                                              
EE                                                   0.92                                              
SI                                                   0.81                                              
DE                                                   0.77                                              
MT                                                   0.47                                              
PL                                                   0.47                                              
FR                                                   0.44                                              
BE                                                   0.33                                              
IE                                                   0.28                                              
DK                                                   0.25                                              
FI                                                   0.25                                              
SE                                                   0.19                                              
ES                                                   0.17                                              
NL                                                   0.16                                              
PT                                                   0.01                                              
HR                                                   0.01                                              
LU                                                   0.01                                              
AT                                                   0.01                                              
CY                                                   0.01                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
IT                                                   6.22               \
SK                                                   1.84                
CZ                                                   5.95                
HU                                                   3.63                
GR                                                   0.87                
BG                                                   1.80                
RO                                                   3.82                
LT                                                   4.59                
LV                                                   2.58        

## 2011 data

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
# ---------------------------------------------------------
dir_path = "../data/exiobase/_ixi/IOT_2011_ixi/"


# ---------------------------------------------------------
# IMPORT MATRICES
# ---------------------------------------------------------
# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL
# ---------------------------------------------------------
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()


In [3]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


Index(['Manure treatment (biogas), storage and land application',
       'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Extraction, liquefaction, and regasification of other petroleum and gaseous materials',
       'Production of electricity by gas',
       'Manufacture of gas; distribution of gaseous fuels through mains',
       'Biogasification of food waste, incl. land application',
       'Biogasification of paper, incl. land application',
       'Biogasification of sewage slugde, incl. land application'],
      dtype='object', name='sector')

In [4]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Extraction of natural gas and services related to natural gas extraction, excluding surveying",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Extraction, liquefaction, and regasification of other petroleum and gaseous materials",
    "Transport via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


### Inter industry

In [5]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)

total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Z)

res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   10,227.76
        Manufacture of gas; distribution of gaseous fuels through mains                                      7.53
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials                0.00
        Transport via pipelines                                                                            631.60
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials          

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
IT                                                  52.02                                             \
CZ                                                  14.39                                              
HU                                                  13.14                                              
LT                                                   6.48                                              
BG                                                   6.18                                              
LV                                                   1.93                                              
FI                                                   1.80                                              
EE                                                   1.20                                              
FR                                                   1.13                                              
GR                                                   0.48                                              
ES                                                   0.25                                              
SE                                                   0.20                                              
PL                                                   0.17                                              
DE                                                   0.14                                              
BE                                                   0.13                                              
NL                                                   0.10                                              
SI                                                   0.09                                              
DK                                                   0.06                                              
RO                                                   0.03                                              
PT                                                   0.03                                              
SK                                                   0.03                                              
MT                                                   0.02                                              
CY                                                   0.01                                              
AT                                                   0.00                                              
HR                                                   0.00                                              
LU                                                   0.00                                              
IE                                                   0.00                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
IT                                                   9.44               \
CZ                                                   5.14                
HU                                                   5.24                
LT                                                   3.26                
BG                                                   1.38                
LV                                                   1.74                
FI                                                  10.49                
EE                                                   3.19                
FR                                                   5.30        

### Final demand

In [6]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)

total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Y)

res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   3,661.67
        Manufacture of gas; distribution of gaseous fuels through mains                                     3.30
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials               0.00
        Transport via pipelines                                                                             6.25
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials             0

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
SK                                                  32.44                                             \
GR                                                  21.99                                              
FI                                                  19.06                                              
RO                                                  17.09                                              
SI                                                   3.54                                              
FR                                                   1.09                                              
LV                                                   1.03                                              
HU                                                   0.94                                              
IT                                                   0.69                                              
HR                                                   0.56                                              
NL                                                   0.33                                              
DE                                                   0.22                                              
PL                                                   0.20                                              
EE                                                   0.18                                              
ES                                                   0.12                                              
SE                                                   0.10                                              
BE                                                   0.10                                              
MT                                                   0.07                                              
BG                                                   0.06                                              
DK                                                   0.05                                              
LT                                                   0.04                                              
CZ                                                   0.02                                              
CY                                                   0.02                                              
AT                                                   0.01                                              
PT                                                   0.01                                              
IE                                                   0.01                                              
LU                                                   0.01                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
SK                                                   2.62               \
GR                                                   1.90                
FI                                                   6.52                
RO                                                   2.23                
SI                                                   0.34                
FR                                                   7.49                
LV                                                   7.08                
HU                                                   0.98                
IT                                                   4.21        

### Inter industry AND final demand

In [7]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY + FINAL DEMAND)
EU_imports_ZY = (
    Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
    + Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
)
totalEU_imports_ZY = EU_imports_ZY.sum(axis=1)

print(totalEU_imports_ZY)

res = EU_imports_ZY.divide(totalEU_imports_ZY, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   13,889.43
        Manufacture of gas; distribution of gaseous fuels through mains                                     10.83
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials                0.00
        Transport via pipelines                                                                            637.85
dtype: float64
region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials          

region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
IT                                                  38.49                                             \
CZ                                                  10.60                                              
HU                                                   9.92                                              
SK                                                   8.57                                              
FI                                                   6.35                                              
GR                                                   6.15                                              
LT                                                   4.79                                              
BG                                                   4.56                                              
RO                                                   4.53                                              
LV                                                   1.69                                              
FR                                                   1.12                                              
SI                                                   1.00                                              
EE                                                   0.93                                              
ES                                                   0.22                                              
PL                                                   0.18                                              
SE                                                   0.17                                              
DE                                                   0.16                                              
NL                                                   0.16                                              
HR                                                   0.15                                              
BE                                                   0.12                                              
DK                                                   0.05                                              
MT                                                   0.03                                              
PT                                                   0.02                                              
CY                                                   0.01                                              
AT                                                   0.01                                              
IE                                                   0.00                                              
LU                                                   0.00                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
IT                                                   7.85               \
CZ                                                   4.23                
HU                                                   3.94                
SK                                                   3.58                
FI                                                   9.28                
GR                                                   1.02                
LT                                                   4.70                
BG                                                   1.53                
RO                                                   2.22        

# pxp MRIO

## 2019 data

In [3]:
# ---------------------------------------------------------
# SPECIFY PATH
dir_path = "../data/exiobase/_pxp/IOT_2019_pxp/"


# ---------------------------------------------------------
# IMPORT MATRICES

# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL

# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()

In [4]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


Index(['Manure (biogas treatment)',
       'Natural gas and services related to natural gas extraction, excluding surveying',
       'Natural Gas Liquids', 'Gas Coke', 'Motor Gasoline',
       'Aviation Gasoline', 'Gasoline Type Jet Fuel', 'Gas/Diesel Oil',
       'Refinery Gas', 'Liquefied Petroleum Gases (LPG)', 'Biogasoline',
       'Electricity by gas', 'Coke oven gas', 'Blast Furnace Gas',
       'Oxygen Steel Furnace Gas', 'Gas Works Gas', 'Biogas',
       'Distribution services of gaseous fuels through mains',
       'Food waste for treatment: biogasification and land application',
       'Paper waste for treatment: biogasification and land application',
       'Sewage sludge for treatment: biogasification and land application'],
      dtype='object', name='sector')

In [5]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Natural gas and services related to natural gas extraction, excluding surveying",
    "Distribution services of gaseous fuels through mains",
    "Natural Gas Liquids",
    "Transportation services via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


### Inter industry

In [6]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)

total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Z)

res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   10,431.67
        Distribution services of gaseous fuels through mains                                   0.00
        Natural Gas Liquids                                                                    0.00
        Transportation services via pipelines                                                169.99
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                                 0.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
IT                                                  68.17                               \
CZ                                                  13.92                                
HU                                                  11.45                                
LT                                                   2.67                                
LV                                                   2.41                                
EE                                                   1.07                                
SI                                                   0.22                                
BG                                                   0.04                                
GR                                                   0.03                                
SK                                                   0.03                                
RO                                                   0.01                                
FR                                                   0.00                                
MT                                                   0.00                                
CY                                                   0.00                                
SE                                                   0.00                                
PT                                                   0.00                                
PL                                                   0.00                                
NL                                                   0.00                                
DE                                                   0.00                                
FI                                                   0.00                                
LU                                                   0.00                                
DK                                                   0.00                                
ES                                                   0.00                                
IE                                                   0.00                                
BE                                                   0.00                                
HR                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
IT                                                    NaN    \
CZ                                                    NaN     
HU                                                    NaN     
LT                                                    NaN     
LV                                                    NaN     
EE                                                    NaN     
SI                                                    NaN     
BG                                                    NaN     
GR                                                    NaN     
SK                                                    NaN     
RO                                                    NaN     
FR                                                    NaN     
MT                                                    NaN     
CY                                                    NaN     
SE                                                    NaN     
PT                                                    NaN     
PL                                                    NaN     
NL                                     

### Final demand

In [7]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)

total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Y)

res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   3,374.79
        Distribution services of gaseous fuels through mains                                  0.00
        Natural Gas Liquids                                                                   0.00
        Transportation services via pipelines                                                 0.19
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                                 0.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
SK                                                  43.98                               \
GR                                                  23.26                                
BG                                                  18.61                                
RO                                                  10.09                                
SI                                                   2.71                                
HU                                                   0.85                                
IT                                                   0.41                                
LV                                                   0.08                                
CZ                                                   0.01                                
EE                                                   0.00                                
DK                                                   0.00                                
CY                                                   0.00                                
SE                                                   0.00                                
PT                                                   0.00                                
PL                                                   0.00                                
NL                                                   0.00                                
MT                                                   0.00                                
LT                                                   0.00                                
LU                                                   0.00                                
IE                                                   0.00                                
BE                                                   0.00                                
HR                                                   0.00                                
DE                                                   0.00                                
FR                                                   0.00                                
FI                                                   0.00                                
ES                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
SK                                                    NaN    \
GR                                                    NaN     
BG                                                    NaN     
RO                                                    NaN     
SI                                                    NaN     
HU                                                    NaN     
IT                                                    NaN     
LV                                                    NaN     
CZ                                                    NaN     
EE                                                    NaN     
DK                                                    NaN     
CY                                                    NaN     
SE                                                    NaN     
PT                                                    NaN     
PL                                                    NaN     
NL                                                    NaN     
MT                                                    NaN     
LT                                     

### Inter-industry AND Final demand

In [8]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY + FINAL DEMAND)
EU_imports_ZY = (
    Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
    + Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
)
totalEU_imports_ZY = EU_imports_ZY.sum(axis=1)

print(totalEU_imports_ZY)

res = EU_imports_ZY.divide(totalEU_imports_ZY, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   13,806.45
        Distribution services of gaseous fuels through mains                                   0.00
        Natural Gas Liquids                                                                    0.00
        Transportation services via pipelines                                                170.19
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                                 0.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
IT                                                  51.61                               \
SK                                                  10.77                                
CZ                                                  10.52                                
HU                                                   8.86                                
GR                                                   5.71                                
BG                                                   4.58                                
RO                                                   2.47                                
LT                                                   2.01                                
LV                                                   1.84                                
SI                                                   0.82                                
EE                                                   0.81                                
LU                                                   0.00                                
SE                                                   0.00                                
PT                                                   0.00                                
PL                                                   0.00                                
NL                                                   0.00                                
MT                                                   0.00                                
CY                                                   0.00                                
DE                                                   0.00                                
IE                                                   0.00                                
BE                                                   0.00                                
HR                                                   0.00                                
FR                                                   0.00                                
FI                                                   0.00                                
ES                                                   0.00                                
DK                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
IT                                                    NaN    \
SK                                                    NaN     
CZ                                                    NaN     
HU                                                    NaN     
GR                                                    NaN     
BG                                                    NaN     
RO                                                    NaN     
LT                                                    NaN     
LV                                                    NaN     
SI                                                    NaN     
EE                                                    NaN     
LU                                                    NaN     
SE                                                    NaN     
PT                                                    NaN     
PL                                                    NaN     
NL                                                    NaN     
MT                                                    NaN     
CY                                     

## 2011 data

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
dir_path = "../data/exiobase/_pxp/IOT_2011_pxp/"


# ---------------------------------------------------------
# IMPORT MATRICES

# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL

# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()

In [3]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]


Index(['Manure (biogas treatment)',
       'Natural gas and services related to natural gas extraction, excluding surveying',
       'Natural Gas Liquids', 'Gas Coke', 'Motor Gasoline',
       'Aviation Gasoline', 'Gasoline Type Jet Fuel', 'Gas/Diesel Oil',
       'Refinery Gas', 'Liquefied Petroleum Gases (LPG)', 'Biogasoline',
       'Electricity by gas', 'Coke oven gas', 'Blast Furnace Gas',
       'Oxygen Steel Furnace Gas', 'Gas Works Gas', 'Biogas',
       'Distribution services of gaseous fuels through mains',
       'Food waste for treatment: biogasification and land application',
       'Paper waste for treatment: biogasification and land application',
       'Sewage sludge for treatment: biogasification and land application'],
      dtype='object', name='sector')

In [4]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Natural gas and services related to natural gas extraction, excluding surveying",
    "Distribution services of gaseous fuels through mains",
    "Natural Gas Liquids",
    "Transportation services via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


### Inter industry

In [5]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)

total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Z)

res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   11,043.58
        Distribution services of gaseous fuels through mains                                   0.00
        Natural Gas Liquids                                                                    0.00
        Transportation services via pipelines                                                603.66
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                               100.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
IT                                                  53.41                               \
CZ                                                  15.07                                
HU                                                  13.68                                
LT                                                   6.77                                
BG                                                   6.40                                
LV                                                   1.91                                
FI                                                   1.56                                
EE                                                   1.06                                
SI                                                   0.07                                
GR                                                   0.04                                
SK                                                   0.01                                
RO                                                   0.00                                
HR                                                   0.00                                
FR                                                   0.00                                
BE                                                   0.00                                
IE                                                   0.00                                
ES                                                   0.00                                
DK                                                   0.00                                
LU                                                   0.00                                
DE                                                   0.00                                
MT                                                   0.00                                
NL                                                   0.00                                
PL                                                   0.00                                
PT                                                   0.00                                
SE                                                   0.00                                
CY                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
IT                                                    NaN    \
CZ                                                    NaN     
HU                                                    NaN     
LT                                                    NaN     
BG                                                    NaN     
LV                                                    NaN     
FI                                                    NaN     
EE                                                    NaN     
SI                                                    NaN     
GR                                                    NaN     
SK                                                    NaN     
RO                                                    NaN     
HR                                                    NaN     
FR                                                    NaN     
BE                                                    NaN     
IE                                                    NaN     
ES                                                    NaN     
DK                                     

### Final demand

In [6]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)

total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
print(total_EU_imports_Y)

res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   3,946.00
        Distribution services of gaseous fuels through mains                                  0.00
        Natural Gas Liquids                                                                   0.00
        Transportation services via pipelines                                                 4.02
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                               100.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
SK                                                  34.06                               \
GR                                                  22.66                                
FI                                                  19.97                                
RO                                                  17.94                                
SI                                                   3.70                                
HU                                                   0.65                                
HR                                                   0.59                                
IT                                                   0.28                                
LV                                                   0.12                                
CZ                                                   0.01                                
BG                                                   0.01                                
EE                                                   0.00                                
DK                                                   0.00                                
MT                                                   0.00                                
SE                                                   0.00                                
CY                                                   0.00                                
PT                                                   0.00                                
PL                                                   0.00                                
NL                                                   0.00                                
LT                                                   0.00                                
LU                                                   0.00                                
IE                                                   0.00                                
BE                                                   0.00                                
FR                                                   0.00                                
DE                                                   0.00                                
ES                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
SK                                                    NaN    \
GR                                                    NaN     
FI                                                    NaN     
RO                                                    NaN     
SI                                                    NaN     
HU                                                    NaN     
HR                                                    NaN     
IT                                                    NaN     
LV                                                    NaN     
CZ                                                    NaN     
BG                                                    NaN     
EE                                                    NaN     
DK                                                    NaN     
MT                                                    NaN     
SE                                                    NaN     
CY                                                    NaN     
PT                                                    NaN     
PL                                     

### Inter industry AND final demand

In [7]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY + FINAL DEMAND)
EU_imports_ZY = (
    Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
    + Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum()
)
totalEU_imports_ZY = EU_imports_ZY.sum(axis=1)

print(totalEU_imports_ZY)

res = EU_imports_ZY.divide(totalEU_imports_ZY, axis=0).T * 100
print(res.sum())
res.sort_values(RU_nat_gas[0], ascending=False)


region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   14,989.58
        Distribution services of gaseous fuels through mains                                   0.00
        Natural Gas Liquids                                                                    0.00
        Transportation services via pipelines                                                607.68
dtype: float64
region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                               100.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
IT                                                  39.43                               \
CZ                                                  11.10                                
HU                                                  10.25                                
SK                                                   8.97                                
FI                                                   6.41                                
GR                                                   6.00                                
LT                                                   4.99                                
RO                                                   4.73                                
BG                                                   4.72                                
LV                                                   1.44                                
SI                                                   1.03                                
EE                                                   0.78                                
HR                                                   0.16                                
SE                                                   0.00                                
PT                                                   0.00                                
PL                                                   0.00                                
NL                                                   0.00                                
MT                                                   0.00                                
CY                                                   0.00                                
LU                                                   0.00                                
DE                                                   0.00                                
IE                                                   0.00                                
BE                                                   0.00                                
FR                                                   0.00                                
ES                                                   0.00                                
DK                                                   0.00                                
AT                                                   0.00                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
IT                                                    NaN    \
CZ                                                    NaN     
HU                                                    NaN     
SK                                                    NaN     
FI                                                    NaN     
GR                                                    NaN     
LT                                                    NaN     
RO                                                    NaN     
BG                                                    NaN     
LV                                                    NaN     
SI                                                    NaN     
EE                                                    NaN     
HR                                                    NaN     
SE                                                    NaN     
PT                                                    NaN     
PL                                                    NaN     
NL                                                    NaN     
MT                                     

# Comparison with EUROSTATS

![Russian imports](../img/Eurostat_russian_imports.png)

Import of Russian Natural Gas (in TJ). Source: [Eurostats](https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_6448417/default/table?lang=en)
